In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("drive/My Drive/Colab Notebooks/YapayZeka/Odev2")

Rasgele 1000 soru üzerinden bir soruya en benzeyen 5 insan cevabını bulup gerçek cevaba göre top1 ve top5 başarılarını hesapladım.
- Cosmos’daki BERT ve GPT modelleri kullanıldı.
- 2 metnin temsilleri arasındaki açı kullanılarak benzerlik ölçümü yapıldı. (cosine_similarity)

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel, GPT2Tokenizer, GPT2Model
import torch

# Veri setini yükleme
data = pd.read_excel("soru_cevap.xlsx").sample(n=1000, random_state=42)

# Türkçe BERT modelini yükleme
tokenizer_bert = BertTokenizer.from_pretrained("ytu-ce-cosmos/turkish-base-bert-uncased")
model_bert = BertModel.from_pretrained("ytu-ce-cosmos/turkish-base-bert-uncased")

# Türkçe GPT-2 modelini yükleme
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained("ytu-ce-cosmos/turkish-gpt2-large-750m-instruct-v0.1")
model_gpt2 = GPT2Model.from_pretrained("ytu-ce-cosmos/turkish-gpt2-large-750m-instruct-v0.1")

# Metin benzerliği fonksiyonu
def calculate_similarity(text1, text2, tokenizer, model):
    inputs1 = tokenizer(text1, return_tensors="pt", padding=True, truncation=True)
    inputs2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)
    vector1 = torch.mean(outputs1.last_hidden_state, dim=1)
    vector2 = torch.mean(outputs2.last_hidden_state, dim=1)
    similarity = cosine_similarity(vector1, vector2)
    return similarity.item()

# En yakın 5 insan cevabını bulma fonksiyonu
def find_top_5_human_answers(question, human_answers, tokenizer, model):
    similarities = []
    for answer in human_answers:
        similarity = calculate_similarity(question, answer, tokenizer, model)
        similarities.append(similarity)
    top_5_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:5]
    top_5_answers = [human_answers[i] for i in top_5_indices]
    return top_5_answers

# Top1 ve Top5 başarılarını hesaplama
def calculate_accuracy(tokenizer, model):
    top1_correct = 0
    top5_correct = 0

    # Tüm insan cevaplarını al
    human_answers = list(data['insan cevabı'])
    # Her bir satırda dolaş
    for index, row in data.iterrows():
        # Soruyu al
        question = row['soru']

        # En yakın 5 insan cevabını bul
        top_5_human_answers = find_top_5_human_answers(question, human_answers, tokenizer, model)

        # Top 1 doğruluk kontrolü
        top_1_human_answer = top_5_human_answers[0]
        if row['insan cevabı'] == top_1_human_answer:
            top1_correct += 1
            top5_correct += 1
        # Top 5 doğruluk kontrolü
        elif row['insan cevabı'] in top_5_human_answers:
            top5_correct += 1

    # Doğruluk oranlarını hesapla
    top1_accuracy = top1_correct / len(data)
    top5_accuracy = top5_correct / len(data)
    return top1_accuracy, top5_accuracy

# BERT modeli ile başarıyı hesapla
top1_accuracy_bert, top5_accuracy_bert = calculate_accuracy(tokenizer_bert, model_bert)
print("BERT ile Top1 Başarısı:", top1_accuracy_bert)
print("BERT ile Top5 Başarısı:", top5_accuracy_bert)

In [ ]:
# GPT-2 modeli ile başarıyı hesapla
top1_accuracy_gpt2, top5_accuracy_gpt2 = calculate_accuracy(tokenizer_gpt2, model_gpt2)
print("GPT-2 ile Top1 Başarısı:", top1_accuracy_gpt2)
print("GPT-2 ile Top5 Başarısı:", top5_accuracy_gpt2)

1000 soru için çalıştırma çok uzun sürüyor, çıktıyı alamıyorum. 100 soru için devam edelim.

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel, GPT2Tokenizer, GPT2Model
import torch

# Veri setini yükleme
data = pd.read_excel("soru_cevap.xlsx").sample(n=100, random_state=42)

# Türkçe BERT modelini yükleme
tokenizer_bert = BertTokenizer.from_pretrained("ytu-ce-cosmos/turkish-base-bert-uncased")
model_bert = BertModel.from_pretrained("ytu-ce-cosmos/turkish-base-bert-uncased")

# Türkçe GPT-2 modelini yükleme
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained("ytu-ce-cosmos/turkish-gpt2-large-750m-instruct-v0.1")
model_gpt2 = GPT2Model.from_pretrained("ytu-ce-cosmos/turkish-gpt2-large-750m-instruct-v0.1")

# Metin benzerliği fonksiyonu
def calculate_similarity(text1, text2, tokenizer, model):
    inputs1 = tokenizer(text1, return_tensors="pt", padding=True, truncation=True)
    inputs2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)
    vector1 = torch.mean(outputs1.last_hidden_state, dim=1)
    vector2 = torch.mean(outputs2.last_hidden_state, dim=1)
    similarity = cosine_similarity(vector1, vector2)
    return similarity.item()

# En yakın 5 insan cevabını bulma fonksiyonu
def find_top_5_human_answers(question, human_answers, tokenizer, model):
    similarities = []
    for answer in human_answers:
        similarity = calculate_similarity(question, answer, tokenizer, model)
        similarities.append(similarity)
    top_5_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:5]
    top_5_answers = [human_answers[i] for i in top_5_indices]
    return top_5_answers

# Top1 ve Top5 başarılarını hesaplama
def calculate_accuracy(tokenizer, model):
    top1_correct = 0
    top5_correct = 0

    # Tüm insan cevaplarını al
    human_answers = list(data['insan cevabı'])
    # Her bir satırda dolaş
    for index, row in data.iterrows():
        # Soruyu al
        question = row['soru']

        # En yakın 5 insan cevabını bul
        top_5_human_answers = find_top_5_human_answers(question, human_answers, tokenizer, model)

        # Top 1 doğruluk kontrolü
        top_1_human_answer = top_5_human_answers[0]
        if row['insan cevabı'] == top_1_human_answer:
            top1_correct += 1
            top5_correct += 1
        # Top 5 doğruluk kontrolü
        elif row['insan cevabı'] in top_5_human_answers:
            top5_correct += 1

    # Doğruluk oranlarını hesapla
    top1_accuracy = top1_correct / len(data)
    top5_accuracy = top5_correct / len(data)
    return top1_accuracy, top5_accuracy

# BERT modeli ile başarıyı hesapla
top1_accuracy_bert, top5_accuracy_bert = calculate_accuracy(tokenizer_bert, model_bert)
print("BERT ile Top1 Başarısı:", top1_accuracy_bert)
print("BERT ile Top5 Başarısı:", top5_accuracy_bert)

BERT ile Top1 Başarısı: 0.6
BERT ile Top5 Başarısı: 0.81


In [ ]:
# GPT-2 modeli ile başarıyı hesapla
top1_accuracy_gpt2, top5_accuracy_gpt2 = calculate_accuracy(tokenizer_gpt2, model_gpt2)
print("GPT-2 ile Top1 Başarısı:", top1_accuracy_gpt2)
print("GPT-2 ile Top5 Başarısı:", top5_accuracy_gpt2)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Rasgele 1000 soru üzerinden bir soruya en benzeyen 5 makine cevabını bulup gerçek cevaba göre top1 ve top5 başarılarını hesapladım.
- Cosmos’daki BERT ve GPT modelleri kullanıldı.
- 2 metnin temsilleri arasındaki açı kullanılarak benzerlik ölçümü yapıldı. (cosine_similarity)

In [ ]:
# En yakın 5 makine cevabını bulma fonksiyonu
def find_top_5_machine_answers(question, machine_answers, tokenizer, model):
    similarities = []
    for answer in machine_answers:
        similarity = calculate_similarity(question, answer, tokenizer, model)
        similarities.append(similarity)
    top_5_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:5]
    top_5_answers = [machine_answers[i] for i in top_5_indices]
    return top_5_answers

# Top1 ve Top5 başarılarını hesaplama (makine cevapları için)
def calculate_machine_accuracy(tokenizer, model):
    top1_correct = 0
    top5_correct = 0

    # Tüm makine cevaplarını al
    machine_answers = list(data['makine cevabı'])
    # Her bir satırı dolaş
    for index, row in data.iterrows():
        question = row['soru']
        top_5_machine_answers = find_top_5_machine_answers(question, machine_answers, tokenizer, model)

        # Top 1 doğruluk kontrolü
        top_1_machine_answer = top_5_machine_answers[0]
        if row['makine cevabı'] == top_1_machine_answer:
            top1_correct += 1
            top5_correct += 1
        # Top 5 doğruluk kontrolü
        elif row['makine cevabı'] in top_5_machine_answers:
            top5_correct += 1
    top1_accuracy = top1_correct / len(data)
    top5_accuracy = top5_correct / len(data)
    return top1_accuracy, top5_accuracy

# BERT modeli ile başarıyı hesapla (makine cevapları için)
top1_accuracy_bert_machine, top5_accuracy_bert_machine = calculate_machine_accuracy(tokenizer_bert, model_bert)
print("BERT ile Makine Cevapları İçin Top1 Başarısı:", top1_accuracy_bert_machine)
print("BERT ile Makine Cevapları İçin Top5 Başarısı:", top5_accuracy_bert_machine)

# GPT-2 modeli ile başarıyı hesapla (makine cevapları için)
top1_accuracy_gpt2_machine, top5_accuracy_gpt2_machine = calculate_machine_accuracy(tokenizer_gpt2, model_gpt2)
print("GPT-2 ile Makine Cevapları İçin Top1 Başarısı:", top1_accuracy_gpt2_machine)
print("GPT-2 ile Makine Cevapları İçin Top5 Başarısı:", top5_accuracy_gpt2_machine)

Rasgele 1000 soru üzerinden bir insan cevabına en benzeyen 5 soruyu bulup gerçek soruya göre top1 ve top5 başarılarını hesapladım.
- Cosmos’daki BERT ve GPT modelleri kullanıldı.
- 2 metnin temsilleri arasındaki açı kullanılarak benzerlik ölçümü yapıldı. (cosine_similarity)

In [ ]:
# En yakın 5 soruyu bulma fonksiyonu
def find_top_5_questions(answer, questions, tokenizer, model):
    similarities = []
    for question in questions:
        similarity = calculate_similarity(question, answer, tokenizer, model)
        similarities.append(similarity)
    top_5_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:5]
    top_5_questions = [questions[i] for i in top_5_indices]
    return top_5_questions

# Top1 ve Top5 başarılarını hesaplama (insan cevaplarına göre)
def calculate_question_accuracy(tokenizer, model):
    top1_correct = 0
    top5_correct = 0

    # Tüm soruları al
    questions = list(data['soru'])
    # Her bir satırı dolaş
    for index, row in data.iterrows():
        answer = row['insan cevabı']
        top_5_questions = find_top_5_questions(answer, questions, tokenizer, model)

        # Top 1 doğruluk kontrolü
        if row['soru'] == top_5_questions[0]:
            top1_correct += 1

        # Top 5 doğruluk kontrolü
        if row['soru'] in top_5_questions:
            top5_correct += 1
    top1_accuracy = top1_correct / len(data)
    top5_accuracy = top5_correct / len(data)
    return top1_accuracy, top5_accuracy

# BERT modeli ile başarıyı hesapla (insan cevaplarına göre)
top1_accuracy_bert_question, top5_accuracy_bert_question = calculate_question_accuracy(tokenizer_bert, model_bert)
print("BERT ile İnsan Cevaplarına Göre Top1 Başarısı:", top1_accuracy_bert_question)
print("BERT ile İnsan Cevaplarına Göre Top5 Başarısı:", top5_accuracy_bert_question)

# GPT-2 modeli ile başarıyı hesapla (insan cevaplarına göre)
top1_accuracy_gpt2_question, top5_accuracy_gpt2_question = calculate_question_accuracy(tokenizer_gpt2, model_gpt2)
print("GPT-2 ile İnsan Cevaplarına Göre Top1 Başarısı:", top1_accuracy_gpt2_question)
print("GPT-2 ile İnsan Cevaplarına Göre Top5 Başarısı:", top5_accuracy_gpt2_question)

Rasgele 1000 soru üzerinden bir makine cevabına en benzeyen 5 soruyu bulup gerçek soruya göre top1 ve top5 başarılarını hesapladım.
- Cosmos’daki BERT ve GPT modelleri kullanıldı.
- 2 metnin temsilleri arasındaki açı kullanılarak benzerlik ölçümü yapıldı. (cosine_similarity)

In [ ]:
# En yakın 5 soruyu bulma fonksiyonu (makine cevapları için)
def find_top_5_questions(answer, questions, tokenizer, model):
    similarities = []
    for question in questions:
        similarity = calculate_similarity(question, answer, tokenizer, model)
        similarities.append(similarity)
    top_5_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:5]
    top_5_questions = [questions[i] for i in top_5_indices]
    return top_5_questions

# Top1 ve Top5 başarılarını hesaplama (makine cevapları için)
def calculate_question_accuracy_machine(tokenizer, model):
    top1_correct = 0
    top5_correct = 0

    # Tüm soruları al
    questions = list(data['soru'])
    # Her bir satırı dolaş
    for index, row in data.iterrows():
        answer = row['makine cevabı']
        top_5_questions = find_top_5_questions(answer, questions, tokenizer, model)

        # Top 1 doğruluk kontrolü
        if row['soru'] == top_5_questions[0]:
            top1_correct += 1

        # Top 5 doğruluk kontrolü
        if row['soru'] in top_5_questions:
            top5_correct += 1
    top1_accuracy = top1_correct / len(data)
    top5_accuracy = top5_correct / len(data)
    return top1_accuracy, top5_accuracy

# BERT modeli ile başarıyı hesapla (makine cevapları için)
top1_accuracy_bert_question, top5_accuracy_bert_question = calculate_question_accuracy(tokenizer_bert, model_bert)
print("BERT ile Makine Cevaplarına Göre Top1 Başarısı:", top1_accuracy_bert_question)
print("BERT ile Makine Cevaplarına Göre Top5 Başarısı:", top5_accuracy_bert_question)

# GPT-2 modeli ile başarıyı hesapla (makine cevapları için)
top1_accuracy_gpt2_question, top5_accuracy_gpt2_question = calculate_question_accuracy(tokenizer_gpt2, model_gpt2)
print("GPT-2 ile Makine Cevaplarına Göre Top1 Başarısı:", top1_accuracy_gpt2_question)
print("GPT-2 ile Makine Cevaplarına Göre Top5 Başarısı:", top5_accuracy_gpt2_question)

Rasgele 1000 soru üzerinden:
Bir soruya en benzeyen 5 insan cevabını bulup gerçek cevaba göre top1 ve top5 başarılarını hesapladık.
Bir soruya en benzeyen 5 makine cevabını bulup gerçek cevaba göre top1 ve top5 başarılarını hesapladık.
Bir insan cevabına en benzeyen 5 soruyu bulup gerçek soruya göre top1 ve top5 başarılarını hesapladık.
Bir makine cevabına en benzeyen 5 soruyu bulup gerçek soruya göre top1 ve top5 başarılarını hesapladık.

Her birinde de BERT ve GPT ile top 1 ve top 5 başarıları  hesapladık.

Şimdi ise soru, insan cevabı, makine cevabı bu temsiller üzerine tsne uygulanarak 2 boyutta türlerine göre renklendirilerek görselleştirelim.
Tsne görselleştirilmesinde bert vektörleri için ayrı, gpt vektörleri için ayrı görsel oluşturalım. Ve türlerine göre renklendirmekte istenilen "soru", "insan cevabı" ve "makine cevabı" sütünları için farklı renkler atayalım.
Yani 2 farklı görsel yapalım, 3 ayrı renk olsun her birinde.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Veri setini yükleme
data = pd.read_excel("soru_cevap.xlsx").sample(n=1000, random_state=42)

# BERT ve GPT modeliyle vektörleri çıkaralım
bert_vectors = []
gpt_vectors = []

for index, row in data.iterrows():
    question = row['soru']
    human_answer = row['insan cevabı']
    machine_answer = row['makine cevabı']

    # BERT modelinden vektörleri çıkar
    inputs_bert = tokenizer_bert(question, human_answer, machine_answer, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs_bert = model_bert(**inputs_bert)
    bert_vector = torch.mean(outputs_bert.last_hidden_state, dim=1).squeeze().numpy()
    bert_vectors.append(bert_vector)

    # GPT-2 modelinden vektörleri çıkar
    inputs_gpt2 = tokenizer_gpt2(question, human_answer, machine_answer, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs_gpt2 = model_gpt2(**inputs_gpt2)
    gpt2_vector = torch.mean(outputs_gpt2.last_hidden_state, dim=1).squeeze().numpy()
    gpt_vectors.append(gpt2_vector)

# TSNE ile boyut indirgeme
tsne = TSNE(n_components=2, random_state=42)
bert_tsne = tsne.fit_transform(bert_vectors)
gpt_tsne = tsne.fit_transform(gpt_vectors)

# Veri çerçevesine TSNE sonuçlarını ekleme
data['bert_tsne_x'] = bert_tsne[:, 0]
data['bert_tsne_y'] = bert_tsne[:, 1]
data['gpt_tsne_x'] = gpt_tsne[:, 0]
data['gpt_tsne_y'] = gpt_tsne[:, 1]

# Görselleştirme
plt.figure(figsize=(10, 5))

# BERT TSNE görseli
plt.subplot(1, 2, 1)
plt.scatter(data['bert_tsne_x'], data['bert_tsne_y'], c=data['soru'], cmap='viridis', label='Soru')
plt.scatter(data['bert_tsne_x'], data['bert_tsne_y'], c=data['insan cevabı'], cmap='plasma', label='İnsan Cevabı')
plt.scatter(data['bert_tsne_x'], data['bert_tsne_y'], c=data['makine cevabı'], cmap='inferno', label='Makine Cevabı')
plt.title('BERT TSNE Görseli')
plt.xlabel('TSNE X')
plt.ylabel('TSNE Y')
plt.legend()

# GPT-2 TSNE görseli
plt.subplot(1, 2, 2)
plt.scatter(data['gpt_tsne_x'], data['gpt_tsne_y'], c=data['soru'], cmap='viridis', label='Soru')
plt.scatter(data['gpt_tsne_x'], data['gpt_tsne_y'], c=data['insan cevabı'], cmap='plasma', label='İnsan Cevabı')
plt.scatter(data['gpt_tsne_x'], data['gpt_tsne_y'], c=data['makine cevabı'], cmap='inferno', label='Makine Cevabı')
plt.title('GPT-2 TSNE Görseli')
plt.xlabel('TSNE X')
plt.ylabel('TSNE Y')
plt.legend()

plt.tight_layout()
plt.show()